# Evaluate Model

## SSD

In [1]:
!pip install -q tqdm

In [19]:
# Imports
import os
import tqdm
import numpy as np
import torch
import torch.nn.functional as F
from config import opt
from core.ssd import SSD
from core.multibox_encoder import MultiBoxEncoder
from core.voc_dataset import VOCDetection
from core.voc_dataset import VOC_LABELS
from core.augmentations import preproc_for_test
from core.utils import detect
from core.voc_eval import voc_eval

In [17]:
# Constantes
MODEL_SSD_FILE = "./modelo/modelo-final.pth"
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
OUTPUT_DIR = "./output"

# Diretório de imagens de treino
voc_root = opt.VOC_ROOT

# Join das imagens
annopath = os.path.join(voc_root, 'VOCtrainval2007', 'VOC2007', 'Annotations', "%s.xml")  
imgpath = os.path.join(voc_root, 'VOCtrainval2007', 'VOC2007', 'JPEGImages', '%s.jpg')    
imgsetpath = os.path.join(voc_root, 'VOCtrainval2007', 'VOC2007', 'ImageSets', 'Main', '{:s}.txt')   
cachedir = os.path.join( os.getcwd(), 'annotations_cache')  

In [4]:
# Instanciar modelo
model_ssd = SSD(opt.num_classes, opt.anchor_num)

In [5]:
# Carregar estado do dicionario
state_dict = torch.load(MODEL_SSD_FILE, map_location=None if torch.cuda.is_available() else 'cpu')

In [6]:
# Carrega o modelo
model_ssd.load_state_dict(state_dict)
model_ssd.to(DEVICE)
print('Modelo Carregado')

Modelo Carregado


In [7]:
# Avaliação do modelo
model_ssd.eval()
multibox_encoder = MultiBoxEncoder(opt)

In [8]:
# Imagens de teste
image_sets = [['2007', 'test']]
test_dataset = VOCDetection(opt, image_sets = image_sets, is_train = False)

In [9]:
files = [open(os.path.join(OUTPUT_DIR, '{:s}.txt'.format(label)), mode = 'w')
        for label in [VOC_LABELS]]

In [10]:
# Loop
for i in tqdm.tqdm(range(len(test_dataset))):
    src = test_dataset[i][0]
        
    img_name = os.path.basename(test_dataset.ids[i][0]).split('.')[0]
    image = preproc_for_test(src, opt.min_size, opt.mean)
    image = torch.from_numpy(image).to(DEVICE)
    
    with torch.no_grad():
        loc, conf = model_ssd(image.unsqueeze(0))
    
    loc = loc[0]
    conf = conf[0]
    conf = F.softmax(conf, dim=1)
    conf = conf.cpu().numpy()
    loc = loc.cpu().numpy()
    
    decode_loc = multibox_encoder.decode(loc)
    gt_boxes, gt_confs, gt_labels = detect(decode_loc, conf, nms_threshold = 0.5, gt_threshold = 0.01)
    
    # Nenhum objecto detectado
    if len(gt_boxes) == 0:
        continue
    
    h, w = src.shape[:2]
    gt_boxes[:, 0] = gt_boxes[:, 0] * w
    gt_boxes[:, 1] = gt_boxes[:, 1] * h
    gt_boxes[:, 2] = gt_boxes[:, 2] * w
    gt_boxes[:, 3] = gt_boxes[:, 3] * h
    
    for box, label, score in zip(gt_boxes, gt_labels, gt_confs):
        print(img_name, "{:.3f}".format(score), "{:.1f} {:.1f} {:.1f} {:.1f}".format(*box), file = files[label])

100%|██████████████████████████████████████████████████████████████████████| 133/133 [01:28<00:00,  1.51it/s]


In [12]:
for f in files:
    print("Fechando arquivo", f.name)
    f.close()

Fechando arquivo ssd/output/pothole.txt


In [20]:
print('Calcula MAP.........')
aps = []
for f in os.listdir(OUTPUT_DIR):
    filename = os.path.join(OUTPUT_DIR, f)
    class_name = f.split('.')[0]
    rec, prec, ap = voc_eval(filename, annopath, imgsetpath.format('test'), class_name, cachedir, ovthresh = 0.1, use_07_metric = True)
    print(class_name, ap)
    aps.append(ap)

print('MAP Médio: ', np.mean(aps))

Calcula MAP.........
pothole 0.7359894920909409
MAP Médio:  0.7359894920909409
